# Files re-distribution

The goal is to re-distribute files within the folder by extension.
For instance, we may have a lot of files in the _Download folder_ and this app can prepare it for cleaning by redistributing it into the folders related to the extension name.

**Simple case** - variable **folders** may contain a list of extensions that should be redistributed

folders = ['pdf', 'jpeg']

**Advanced case** - folder variable may contain a dictionary where the key is the folder name and the value is a list of extensions to consider

folders = {"pdf": ["pdf"], "img": ["jpeg", "jpg", "png", "gif"]}

Let's continue with **Advanced** case

In [1]:
import os
from pathlib import Path

Define target folder

In [2]:
target_folder = r"C:\Users\Admin\Downloads"

Prepare Print utilities for better visibility

In [4]:
# Print Dict key & value
def print_dict(dict_values: dict):
    # sort key
    keys = list(dict_values.keys())
    keys.sort()
    for key in keys:
        print(f'{key}: {dict_values.get(key)}')

# Print list
def print_list(list_values: list):
    for value in list_values:
        print(f'{value}')

Let's see how many different file extensions we have in target folder 

In [5]:
folder = '[folders]'
values = dict()
values[folder] = 0

for item in Path(target_folder).iterdir():
    # folder
    if item.is_dir():
        values[folder] += 1
        continue
    
    # file
    if item.is_file():
        ext = item.suffix[1:]
        if values.get(ext) is None:
            values[ext] = 1
        else:
            values[ext] += 1

In [ ]:
print_dict(values)

Let's define what files (with sufficses) we can re-organize into a new folders. Also let's define exception list if required (files that should be ignored)

In [7]:
# define destination folders with extensions
destination_folders = {
    '_books': ["pdf", "epub"],
    '_zip': ["zip"],
    '_img': ["jpg", "jpeg", "png", "gif", "img"]
}

# define names that should be excluded from the list or keep it empty
name_exception = ['vik']

Utilities

In [8]:
# Check if name in exclusion list
def is_name_in_exception(name: str) -> bool:
    for item in name_exception:
        if name.lower().find(item) != -1:
            return True
    return False


# Get all extensions
def get_all_extensions() -> set:
    ext = []
    for item_list in destination_folders.values():
        ext.extend(item_list)
    return set(ext)

# Get folder by extension
def get_folder_by_extension(sufix: str) -> str | None:
    for folder_name, list_ext in destination_folders.items():
        if sufix in list_ext:
            return folder_name
    return None

Prepare list of files to move

In [9]:
files = dict()
file_exceptions = []

all_extentions = get_all_extensions()

for item in Path(target_folder).iterdir():
    if item.is_dir():
        continue

    ext = item.suffix[1:]
    if item.is_file() and ext in all_extentions:
        if is_name_in_exception(item.name):
            file_exceptions.append(item.name)
            continue

        folder_name = get_folder_by_extension(ext)
        if folder_name is None:
            continue
        if files.get(folder_name) is None:
            files[folder_name] = []
        file_list: list = files[folder_name]
        file_list.append(item)


In [ ]:

print('-- Files to Move --')
print_list(files['_img'])
print('-- Excluded Files --')
print_list(file_exceptions)


Move the files into destination folders

In [11]:
for folder_name, file_list in files.items():
    path_name = target_folder + '\\' + folder_name
    if not os.path.exists(path_name):
        print(f'Create -> {path_name}')
        os.makedirs(path_name)

    moved = 0
    for file in file_list:
        file: Path = file
        destination = Path(path_name + '\\' + file.name)
        file.rename(destination)
        moved += 1

    print(f'Into folder "{folder_name}" moved {moved} file(s)')

Into folder "_img" moved 0 file(s)
Into folder "_zip" moved 0 file(s)
Into folder "_books" moved 0 file(s)
